## Classification Loss:
### 1. Hinge loss or Multi class SVM Loss:
The multi-class SVM loss, also known as the multiclass hinge loss, is a commonly used loss function in machine learning for training support vector machine (SVM) models to perform multi-class classification tasks.

The goal of the multi-class SVM loss is to encourage the correct class score to be higher than the scores of all other classes by a margin. It computes the difference between the scores of the correct class and the scores of all other classes and applies a hinge loss function to penalize any violation of the margin.

$L_i = \sum_{j\neq y_i} \max(0, s_j - s_{y_i} + \Delta)$

$L_i = \sum_{j\neq y_i} \max(0, w_j^T x_i - w_{y_i}^T x_i + \Delta)$

Where
- $x_i$ is the input
- $y_i$ specifies the index of the correct class.
- $s_j = f(x_i, W)_j$

### 2. Cross Entropy Loss / Log Loss / Sofmax Loss :



